<a href="https://colab.research.google.com/github/rahulgundala007/NLP_text_summarization/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset('Rahulgundala007/ToSdataset2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import AutoTokenizer

model_name = 't5-small'  # You can choose other versions of T5 like 't5-base', 't5-large', etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    # Prefix input text with 'summarize: '
    inputs = ["summarize: " + doc for doc in examples['ToS_Detail']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # No prefix for summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['ToS_Summary'], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
dataset = dataset.map(preprocess_data, batched=True)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/65 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [5]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    save_strategy="epoch",           # Save the model at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    weight_decay=0.01,               # Weight decay for regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],   # Use your dataset splits appropriately
    eval_dataset=dataset['validation'] # Use your dataset splits appropriately
)

trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.671835
2,No log,2.648273
3,No log,2.296098


TrainOutput(global_step=51, training_loss=4.88160915000766, metrics={'train_runtime': 587.6335, 'train_samples_per_second': 0.332, 'train_steps_per_second': 0.087, 'total_flos': 26391651287040.0, 'train_loss': 4.88160915000766, 'epoch': 3.0})

In [6]:
model.save_pretrained('./my_t5_summarizer')
tokenizer.save_pretrained('./my_t5_summarizer')


('./my_t5_summarizer/tokenizer_config.json',
 './my_t5_summarizer/special_tokens_map.json',
 './my_t5_summarizer/spiece.model',
 './my_t5_summarizer/added_tokens.json',
 './my_t5_summarizer/tokenizer.json')

**Epochs:** The training ran for 3 epochs. An epoch is a full pass through your entire training dataset.

**Training Loss**: This indicates how well the model is fitting the training data. Lower values mean better fit. "No log" suggests that the loss wasn't logged during the training but only at the end of each epoch for validation.

**Validation Loss**: Represents the model's performance on a separate part of the dataset not seen during training (the validation set). It's crucial for assessing how well the model generalizes to new, unseen data. Like training loss, lower is better. The validation loss decreased from 4.671835 to 2.296098, indicating that the model's predictions became more accurate with respect to the validation set as training progressed.

**global_step**: The total number of training steps completed. A step is a single gradient update; this number is 51, indicating your model's weights were updated 51 times.

**training_loss:** The average loss over all training steps. Here, it's somewhat higher than your final validation loss, which is typical as the model learns from the training data.

**metrics:** This dictionary contains various performance metrics:

**train_runtime:** Total training time in seconds (605.114).

**train_samples_per_second:** The number of training samples processed per second (0.322).

**train_steps_per_second**:The number of training steps (weight updates) performed per second (0.084).

**total_flos:** The total number of floating-point operations performed during training. This gives an idea of the computational cost (26,391,651,287,040 FLOPs).

**train_loss:** Repetition of the training loss.

**epoch**: The final epoch number (3.0).

In [7]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the model and tokenizer
model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Prepare input text
input_text = "summarize: This is a long detailed text that you want to summarize. It may contain multiple sentences covering different aspects of a particular topic or subject matter."
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
output_tokens = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=150,
    num_beams=5,
    early_stopping=True
)

# Decode and print the summary
summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(summary)


this is a long detailed text that you want to summarize. it may contain multiple sentences covering different aspects of a particular topic or subject matter.


In [8]:
pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.2 MB/s eta 0:00:00


In [9]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_tokens = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=150,
        num_beams=5,
        early_stopping=True
    )
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return summary


In [10]:
pip install PyPDF2 transformers sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [11]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:  # Ensure there is text on the page
                text += page_text + ' '  # Append text of each page and a space
    return text


In [17]:
# Specify the path to your PDF file
pdf_path = '/content/google_terms_of_service_en_us.pdf'

# Call the function with the PDF path
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)


GOOGLE TERMS OF SERVICE
Effective January 5, 2022
Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of Service, but
it’s impo ant to establish what you can expect from us as
you use Google services, and what we expect from you.
These Terms of Service re ect the way Google’s business works, the laws that apply to our company, and
certain things we’ve always believed to be true. As a result, these Terms of Service help de ne Google’s
relationship with you as you interact with our services. For example, these terms include the following topic
headings:
What you can expect from us, which describes how we provide and develop our services
What we expect from you, which establishes certain rules for using our services
Content in Google services, which describes the intellectual property rights to the content you  nd in our
services — whether that content belongs to you, Google, or others
In case of problems or disagreements, which describes other legal 

In [21]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_tokens = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=1000,  # Adjust based on expected summary length
        min_length=500,    # Ensure the summary is at least 500 tokens
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2,  # Prevents the model from repeating the same n-grams
        length_penalty=1.0,
    )
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return summary



In [22]:
import sacrebleu

def compute_bleu(generated_summary, reference_summary):
    bleu_score = sacrebleu.corpus_bleu([generated_summary], [[reference_summary]])
    return bleu_score.score


In [25]:
pdf_path = '/content/google_terms_of_service_en_us.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
generated_summary = generate_summary(pdf_text)
print("Generated Summary:", generated_summary)

# You must provide this
reference_summary = "Terms of Service for Google As of January 5, 2022 Versions from the past What these phrases encompass. These Terms of Service reflect the business practices of Google, the laws that govern our operation, and certain long-held beliefs. The following subject headings are included in this: What our expectations are, outlining specific guidelines for utilizing our services Google services' content. In addition, we make our privacy policy available. We recommend that you read it to...G.A., if you are in place, this indicates that you and your relationship with the e.g. (s) and what we are covered by these Terms; nevertheless, they also aid in defining how you engage with Google and certain terms and services, such as the following: what You should read them to better understand your rights and those of your child. When you use these services to de? the, so that we can't and will, for example, read this to get your permission to do so, in case you are, how your & how the content is in and out of this, as we ; and when, use, your consent. What you want to see from our — and how we, or what they mean — what our customers and our content, what to expect in the case of problems and other, to understand how to use Google to help you with your privacy. browser, where you'll be in your favor, allow us to use your data to enhance your access to the data. It's crucial to make sure that your users and/or what your Google and you, that, by using the service and! the (and the other) terms to make it more accessible to you s.c. and, such, be able to change your preferences based on your personal, at the end, with and without the need for permission. Please, please, understand what you would like to hear from us (the) before engaging in this, as it will help us understand why. Regardless, you are free to use our service."
bleu_score = compute_bleu(generated_summary, reference_summary)
print(f"BLEU score: {bleu_score}")


Generated Summary: GOOGLE TERMS OF SERVICE Effective January 5, 2022 Archived versions What’s covered in these terms. These Terms of Service reect the way Google's business works, the laws that apply to our company, and certain things we’ve always believed to be true. This includes the following topic headings: What you can expect from us, which establishes certain rules for using our services Content in Google services. Besides this, we also publish a Privacy Policy. We encourage you to read it to ... 'G.A., if you’re in place, this means that you and your relationship with the e.g. (s) and what we're covered by these Terms, but they help dene the how you interact with Google - and some of the terms and services, including the: what you want to see from our — and how we, or what they mean – what our customers and our content, what to expect in the case of problems and other, to understand how to use the Google to help you with your privacy and the information you need to know to learn